In [1]:
import plotly
import plotly.graph_objs as go
import pandas as pd

from plotly import tools

plotly.offline.init_notebook_mode(connected=True)

In [2]:
group_by = 'num_groupe_etudiant'
reindex = True
ascending = False

In [16]:
colorscale = [[0, 'rgb(178, 10, 28)'],[1, 'rgb(129, 201, 159)']]
colorbar = {'lenmode':'pixels','len':200, 'yanchor':'top', 'y':1, 'dtick':1}

In [11]:
df_students = pd.read_csv('ExportElevesUVSimple.csv')
df_students.columns = [x.lower() for x in df_students.columns]
df_students['niveau_atteint_max_francais'].fillna('Maternel', inplace=True)

In [12]:
group_variables = ['id_eleve','groupe_promo','num_groupe_etudiant','libelle','site',
                    'code_formation','code_admission','niveau_atteint_max_francais']

In [13]:
df = pd.read_csv('QCM.csv')
df.columns = [x.lower() for x in df.columns]

In [22]:
df.head()

,nom,id_eleve,bus3,clef2,clef5,composants16,composants3,conforme3,dashboard1,degeneree1,...,redondance2,schema2,scrum2,groupe_promo,num_groupe_etudiant,libelle,site,code_formation,code_admission,niveau_atteint_max_francais
41,Storm Katana,4579,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1,1,DDRSE,Brest,IG 3A,ST CONV,Maternel
42,Sunset Lionchanter,3095,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1,1,DDRSE,Brest,MSC 2A DS,ADO,A2
44,Tempest Queensong,14462,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1,1,DDRSE,Brest,IG 3A,CC MP,Maternel
47,Tome Starfinder,13046,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1,1,DDRSE,Brest,IG 3A,CC MP,Maternel
49,Typhoon Honor,13089,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1,1,DDRSE,Brest,IG 3A,CC MP,B2


In [14]:
student_id = 'id_eleve'
student_name = 'nom'
observations = [x for x in list(df.columns) if x not in [student_id, student_name]]

df = df.merge(df_students[group_variables], on='id_eleve')
if reindex:
    df.reindex(sorted(df.columns), axis=1)

df.sort_values(by=student_name, inplace=True, ascending=ascending)

In [1]:
df.sort_values(by=[group_by,student_name], inplace=True)
values = df[group_by].unique() #num_groupe_etudiant

fig = tools.make_subplots(rows=len(values), cols=1, shared_xaxes=False)
height = len(df)*25+(len(values))*25
ymin = height

for row, group in enumerate([x for x in values]):
    df_group = df[df[group_by] == group]
    df_group.sort_values(by=student_name, inplace=True, ascending=ascending)
    
    ymax = ymin - len(df_group) * 20

    if row==0:
        trace = go.Heatmap(z=df_group[observations].values.tolist(), 
                           x=observations, y=df_group[student_name].values.tolist(),
                           ygap=2, xgap=1, 
                           colorscale=colorscale, showscale=True)
    else:
        trace = go.Heatmap(z=df_group[observations].values.tolist(), 
                           x=observations, y=df_group[student_name].values.tolist(),
                           ygap=2, xgap=1, 
                           colorscale=colorscale, showscale=False)
        
    fig.append_trace(trace,row+1, 1)
    
    label_xkey = 'xaxis{}'.format(row+1)
    label_ykey = 'yaxis{}'.format(row+1)
    label_value = str(group)
      
    if row==0:
        fig['layout'].update({label_xkey:{'showgrid':False},
                              label_ykey:{'title':label_value,
                                          'domain':[ymax/height, ymin/height],
                                          'showgrid':False}})
    else:
        fig['layout'].update({label_xkey:{'showticklabels':False,'ticks':'','showgrid':False},
                              label_ykey:{'title':label_value,
                                          'domain':[ymax/height, ymin/height],
                                          'showgrid':False}})
    ymin = ymax - 20

layout = {'autosize':True,
          'height':50+height,
          'width':200+len(observations)*20,
          'margin':go.layout.Margin(l=200,t=100),
          'xaxis1': {'side':'top'},
          'paper_bgcolor':'rgb(239, 239, 239)'}
    
fig['layout'].update(layout)
fig['data'][0]['colorbar'] = colorbar
plotly.offline.iplot(fig, filename='heatmap')

NameError: name 'df' is not defined